# Import packages et fichiers sources

In [5]:
#import des bilbiothèques nécessaires
import pandas as pd
import os
from itertools import permutations
import requests
import json
import polyline
import geopandas as gpd
import folium
import numpy as np
import time  
from shapely.geometry import LineString, Point
from shapely.wkt import loads as wkt_loads

# Afficher les versions pour vérifier que tout est bien installé
print("requests:", requests.__version__)
print("pandas:", pd.__version__)
print("geopandas:", gpd.__version__)
print("folium:", folium.__version__)
print("numpy:", np.__version__)

requests: 2.28.1
pandas: 2.2.3
geopandas: 1.0.1
folium: 0.19.5
numpy: 1.26.4


In [6]:
# Charger la liste des centroides des quartiers depuis un CSV (colonnes id-nom-latitude-longitude)
#df_lieux_csv = pd.read_csv("coords_quartiers_test.csv")
df_lieux_csv = pd.read_csv("input/coords_quartiers.csv")

#Charge la matrice de demande OD entre tous les quartiers (Attention à la bonne correspondance des noms)
file_path = "input/9000_2410-tab-gpe-TIM-2030PPS_OD_85quartiers_6corridors_by_Script.xlsx"  
df_matrice_OD = pd.read_excel(file_path, sheet_name="NbDépVP", index_col=0)

In [7]:
# Créer toutes les paires possibles sans les paires origine = destination
paires_lieux = list(permutations(df_lieux_csv.itertuples(index=False), 2))  # (85+6)*(85+6) - (85+6) = 8190 paires

# Clef d'appel API Google Maps (directions)

In [8]:
# doc ici : https://developers.google.com/maps/documentation/directions/overview?hl=fr
Clef = 'YOUR_API_KEY'

# Fonctions cache - sécurité

#### Les fonctions ci-dessous permettent de stocker l'information si une paire OD a deja été investiguée dans un cache. Si c'est le cas, l'API ne fera pas d'appel et cela permet d'éviter des appels inutiles

In [9]:
#fonction pour rajouter un itinéraire au cache
#def add_to_cache(origin, destination, route_info, cache, i, cache_file="cache_routes_test.json"):
def add_to_cache(origin, destination, route_info, cache, i, cache_file="cache_routes.json"):
    cache_key = f"{origin[0]}_{destination[0]}"
    cache[cache_key] = route_info  # Ajouter les données au cache
    save_cache(cache, cache_file, i)  # Sauvegarder le cache immédiatement
    
# Charger le cache à partir du fichier JSON (s'il existe)
#def load_cache(cache_file="cache_routes_test.json"):
def load_cache(cache_file="cache_routes.json"):
    try:
        with open(cache_file, "r") as f:
            content = f.read().strip()  # Lire et supprimer les espaces inutiles
            if not content:  # Si le fichier est vide
                return {}  # Retourner un dictionnaire vide
            cache = json.loads(content)
            for key, route in cache.items():
                route["start_location"] = Point(route["start_location"])
                route["end_location"] = Point(route["end_location"])
                route["geometry"] = wkt_loads(route["geometry"])
            return cache  # Charger les données du cache
    except FileNotFoundError:
        with open(cache_file, "w") as f:
            json.dump({}, f)  # Créer un fichier vide si pas existant
        return {}  # Si le fichier n'existe pas encore, retourner un cache vide

# Sauvegarder le cache dans le fichier JSON
def save_cache(cache, cache_file, i):
    attempts = 5  # Nombre de tentatives avant d'abandonner
    interval = 50 #Enregistrer dans le fichier tous les 50 itinéraires (soulage l'ouverture du cache)
    max_itineraire = 8190 #A modifier selon le nombre max d'itinéraires prévus afin d'intégrer le reliquat d'itinéaire (<50) dans le cache
    for j in range(attempts):
        if i % interval == 0 or i == max_itineraire :
            try:
                with open(cache_file, "w") as f:
                    json.dump(cache, f)
                    print("CACHED")
                return  
            except PermissionError:
                print(f"Erreur de permission, tentative {j+1}/{attempts}...")
                time.sleep(1)  # Attendre 1s avant de réessayer d'écrire dans le fichier json
            print("Impossible d'écrire dans le fichier json après plusieurs tentatives.")


# Vérifier si l'itinéraire entre les deux lieux est dans le cache
def is_cached(origin, destination, cache):
    # Créer la clé unique pour la paire de lieux (origin, destination)
    cache_key = f"{origin[0]}_{destination[0]}"
    return cache_key in cache  # Si la clé existe, l'itinéraire est dans le cache

# Récupérer un itinéraire depuis le cache
def get_from_cache(origin, destination, cache):
    cache_key = f"{origin[0]}_{destination[0]}"
    return cache.get(cache_key)  # Retourner les données du cache pour cette clé


# Fonction d'appel API Google Maps (directions)

In [10]:
# Fonction pour récupérer l'itinéraire
def get_route(origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{origin[2]},{origin[3]}",
        "destination": f"{destination[2]},{destination[3]}",
        "mode": "driving",
        "key": Clef
    }
    response = requests.get(url, params=params).json()
    
    # Vérifier s'il y a un itinéraire
    if response["status"] == "OK" : # si succès de la requête
        route = response["routes"][0]
        leg = route["legs"][0]

        # Extraire les données importantes
        distance = leg["distance"]["value"]
        duration = leg["duration"]["value"]
        start_address = leg["start_address"]
        end_address = leg["end_address"]
        encoded_polyline = route["overview_polyline"]["points"]
        decoded_polyline = polyline.decode(encoded_polyline)

        # Créer la géométrie LineString
        line_geom = LineString(decoded_polyline)
        
        return {
            "start_id": origin[0],
            "start_name": origin[1],
            "start_adresse": start_address,
            "start_location": [origin[2], origin[3]],
            "end_id": destination[0],
            "end_name": destination[1],
            "end_adresse": end_address,
            "end_location": [destination[2], destination[3]],
            "distance_m": distance,
            "duration_s": duration,
            "geometry": line_geom.wkt
        }
    else:
        return "Pas d'itinéraire trouvé pour";origin[1];destination[1]  # Retourne None si pas d'itinéraire

In [11]:
# Fonction principale pour récupérer l'itinéraire avec cache ou l'API (via fonction get_route)
def get_route_with_cache(origin, destination, cache, i):
    
    if is_cached(origin, destination, cache):
        print(f"Cache hit: {origin} -> {destination}")
        return get_from_cache(origin, destination, cache)  # Récupérer depuis le cache

    # Sinon, appeler l'API
    print(f"API request: {origin} -> {destination}")
    route_info = get_route(origin, destination)
    
    if route_info:
        add_to_cache(origin, destination, route_info, cache, i)  # Ajouter au cache
    return route_info

In [12]:
#Les coordonnées doivent être en général toujours au format longitude/latitude mais l'API maps donne latitude/longitude donc une fonction d'inversion permet de remttre les choses en ordre
def invert_coordinates(geom):
    if isinstance(geom, LineString):
        return LineString([(y, x) for x, y in geom.coords])  # Inversion (lat, lon) -> (lon, lat)
    if isinstance(geom, Point):
        return Point([(y, x) for x, y in geom.coords])  # Inversion (lat, lon) -> (lon, lat)
    return geom

In [13]:
def get_demand(origin, destination, matrix):
    try:
        return matrix.loc[origin, destination]
    except KeyError:
        return 9999  # Si la paire OD n'existe pas, attribuer 0 --> permettra de trouver les erreurs d'association au besoin

# Boucle sur toutes les paires OD pour récupérer les polylignes

!!! NE PAS RUN SANS ETRE SUR DE LA PERTINENCE : BOUCLE AVEC APPEL API PAYANT !!!

In [14]:
# Charger le cache existant (si disponible)
cache = load_cache()

routes_data = []  # Stocke les itinéraires

remove_chars = [')', '(', "'"]

i = 0 #(compteur des itinéraires appelés)

for origin_str, destination_str in paires_lieux:
    
    # Découper la chaîne pour obtenir les différentes valeurs (passage du dictionnaire tuple en chaine de caractère propre)
    origin_str_converted = f"{origin_str}"
    destination_str_converted = f"{destination_str}"
    for char in remove_chars: #nettoyage de la chaien de caractère
        origin_str_converted = origin_str_converted.replace(char, '')
        destination_str_converted = destination_str_converted.replace(char, '')

    origin_parts = origin_str_converted.split(';')  # crèe 4 champs distinct
    destination_parts = destination_str_converted.split(';')  # idem
    
    
    # Extraire les informations nécessaires
    origin_id = origin_parts[0]
    origin_name = origin_parts[1]
    origin_latitude = float(origin_parts[2])  # Convertir en float pour les coordonnées
    origin_longitude = float(origin_parts[3])
    
    destination_id = destination_parts[0]
    destination_name = destination_parts[1]
    destination_latitude = float(destination_parts[2])
    destination_longitude = float(destination_parts[3])

    # Afficher pour vérifier
#     print(f"Origin: {origin_name} ({origin_latitude}, {origin_longitude})")
#     print(f"Destination: {destination_name} ({destination_latitude}, {destination_longitude})")
    
    i=i+1
    route_info = get_route_with_cache(origin_parts, destination_parts, cache, i) #APPEL API ICI
    
    if route_info:
        routes_data.append(route_info)  # Ajouter les résultats valides
    
    time.sleep(0.5)  # Pause pour éviter de surcharger l'API Google et de bien enregistrer le cache, 2e sécurité en + du cache

for route in routes_data:
    if isinstance(route["geometry"], str):  # Vérifie si c'est une chaîne WKT
        route["geometry"] = wkt_loads(route["geometry"])
    elif not isinstance(route["geometry"], LineString):  # Vérifie qu'on a bien une LineString
        raise TypeError(f"Valeur inattendue pour 'geometry': {route['geometry']}")
        
# Créer le GeoDataFrame final avec correction de la forme des coordonnées (lat/long > long/lat) et association de la demande à chaque paire OD
gdf_routes = gpd.GeoDataFrame(routes_data, geometry="geometry", crs="EPSG:4326")
gdf_routes["geometry"] = gdf_routes["geometry"].apply(invert_coordinates)
gdf_routes["start_location"] = gdf_routes["start_location"].apply(invert_coordinates)
gdf_routes["end_location"] = gdf_routes["end_location"].apply(invert_coordinates)

gdf_routes["demande"] = gdf_routes.apply(lambda row: get_demand(row["start_name"], row["end_name"], df_matrice_OD),axis=1)

print(gdf_routes.head())
print("Fin de création des polylignes")


API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=4', 'Aubert-Rieu', '46.19142229', '6.1641093']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=5', 'Avully', '46.16757397', '5.9945352']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=6', 'Avusy', '46.14958764', '6.0028934']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=7', 'Aïre-Le Lignon', '46.2025076', '6.09762774']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=8', 'Balexert', '46.21963443', '6.11278009']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=9', 'Bardonnex', '46.14666677', '6.12442283']
API request

API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=68', 'Presinge', '46.21752181', '6.25591694']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=69', 'Puplinge', '46.21122678', '6.23067922']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=70', 'Pâquis', '46.21177947', '6.15015277']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=71', 'Russin', '46.19000069', '6.01369358']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=72', 'Satigny', '46.21020664', '6.04225524']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=73', 'Soral', '46.14283852', '6.04233221']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=74', 'St-Jean/Charmilles', '46.20640399', '6.12435193']
API request: ['Pandas_0=1', 'AIG', '46.23270221', '6.11309231'] -> ['Pandas_0=75', 'Sécheron', '46.21831034', '6.14494946']
API re

API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=41', 'G-S', '46.2400542', '6.11957884']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=42', 'Gare-Bachet', '46.17527689', '6.12811173']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=43', 'Gd-Saconnex', '46.23082692', '6.12248403']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=44', 'Genthod', '46.26464665', '6.15282849']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=45', 'Gradelle', '46.20390224', '6.18610119']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=46', 'Grand-Pré', '46.21295839', '6.1277705']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.04209559'] -> ['Pandas_0=47', 'Gy', '46.25223042', '6.25664073']
API request: ['Pandas_0=2', 'Aire-la-Ville', '46.19034489', '6.042095

API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=13', 'Bernex-Campagne', '46.17111846', '6.06979977']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=14', 'Bernex-Nord', '46.1814206', '6.07926999']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=15', 'Bougeries-centre', '46.19768625', '6.18297538']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=16', 'Bougeries-sud', '46.18877566', '6.17683887']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=17', 'Bout-du-Monde', '46.18303219', '6.15683852']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=18', 'CB-Gare', '46.19575517', '6.19837583']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=19', 'CERN', '46.23309441', '6.05446986']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=

API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=77', 'Troinex', '46.16249014', '6.14772937']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=78', 'Vandoeuvres', '46.21734559', '6.20191721']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=79', 'Vernets-Acacias', '46.19214057', '6.13541804']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=80', 'Vernier-Gare', '46.21965306', '6.0958264']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=81', 'Vernier-Village', '46.2117715', '6.07954962']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=82', 'Versoix', '46.28464191', '6.16203048']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=83', 'Veyrier', '46.16976775', '6.16828098']
API request: ['Pandas_0=3', 'Anières', '46.27749176', '6.22791794'] -> ['Pandas_0=84', '

...

API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=69', 'Puplinge', '46.21122678', '6.23067922']
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=70', 'Pâquis', '46.21177947', '6.15015277']
CACHED
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=71', 'Russin', '46.19000069', '6.01369358']
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=72', 'Satigny', '46.21020664', '6.04225524']
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=73', 'Soral', '46.14283852', '6.04233221']
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=74', 'St-Jean/Charmilles', '46.20640399', '6.12435193']
API request: ['Pandas_0=88', 'CorridorAnnecy', '45.90170033', '6.120700651'] -> ['Pandas_0=75', 'Sécheron', '46.21831034', '6.14494946']
API request: ['Pandas_0=88', 'Co

API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=40', 'En-Sauvy', '46.17628826', '6.11907846']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=41', 'G-S', '46.2400542', '6.11957884']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=42', 'Gare-Bachet', '46.17527689', '6.12811173']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=43', 'Gd-Saconnex', '46.23082692', '6.12248403']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=44', 'Genthod', '46.26464665', '6.15282849']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=45', 'Gradelle', '46.20390224', '6.18610119']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492'] -> ['Pandas_0=46', 'Grand-Pré', '46.21295839', '6.1277705']
API request: ['Pandas_0=89', 'Bellegarde', '46.10867441', '5.824570492

API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=12', 'Bernex', '46.17537281', '6.07467559']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=13', 'Bernex-Campagne', '46.17111846', '6.06979977']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=14', 'Bernex-Nord', '46.1814206', '6.07926999']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=15', 'Bougeries-centre', '46.19768625', '6.18297538']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=16', 'Bougeries-sud', '46.18877566', '6.17683887']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=17', 'Bout-du-Monde', '46.18303219', '6.15683852']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=18', 'CB-Gare', '46.19575517', '6.19837583']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.06071609

API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=75', 'Sécheron', '46.21831034', '6.14494946']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=76', 'Thônex', '46.19477286', '6.2053396']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=77', 'Troinex', '46.16249014', '6.14772937']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=78', 'Vandoeuvres', '46.21734559', '6.20191721']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=79', 'Vernets-Acacias', '46.19214057', '6.13541804']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=80', 'Vernier-Gare', '46.21965306', '6.0958264']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['Pandas_0=81', 'Vernier-Village', '46.2117715', '6.07954962']
API request: ['Pandas_0=90', 'PaysGex', '46.33354785', '6.060716091'] -> ['

API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=49', 'Jonction', '46.19945506', '6.13544282']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=50', 'Jussy', '46.2341754', '6.26548713']
CACHED
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=51', 'LPR', '46.18781731', '6.12946891']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=52', 'Laconnex', '46.15631513', '6.03617825']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=53', 'Lancy', '46.1824823', '6.11340498']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=54', 'Lancy-St-Georges', '46.19365195', '6.11918087']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=55', 'Meinier', '46.24565131', '6.23033013']
API request: ['Pandas_0=91', 'Vaud', '46.38432514', '6.236226779'] -> ['Pandas_0=56', 'Meyrin', '46.23444493',

In [15]:
# Sauvegarde en GeoJSON (format léger)
gdf_routes.to_file("output/itineraire_global_TIM.geojson", driver="GeoJSON")
#gdf_routes.to_file("output/itineraire_test_TIM.geojson", driver="GeoJSON")

# Sauvegarde en Shapefile (compatible QGIS/ArcGIS)
gdf_routes.to_file("output/itineraire_global_TIM.shp")
#gdf_routes.to_file("output/itineraire_test_TIM.shp")
print("Fichiers créés")

C:\Users\gpe\AppData\Local\Temp\ipykernel_34128\2027635788.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_routes.to_file("output/itineraire_global_TIM.shp")
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'start_adresse' to 'start_adre'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'start_location' to 'start_loca'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'end_adresse' to 'end_adress'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'end_location' to 'end_locati'
  ogr_write(


Fichiers créés


# Affichage interactif des résultats

In [20]:
# Créer une carte centrée sur le premier lieu
m = folium.Map(location=[gdf_routes.geometry[0].centroid.y, gdf_routes.geometry[0].centroid.x], zoom_start=12)

# Ajouter les itinéraires à la carte
for _, row in gdf_routes.iterrows():
    coords = [(point[1], point[0]) for point in row.geometry.coords]  # (lat, lon), folium utilise latitude en 1er
    folium.PolyLine(coords, color="blue", weight=2, opacity=0.4).add_to(m)

# Afficher la carte
#m #A décommenter pour afficher mais peut faire ramer si beaucoup de lignes à afficher

## Conversion en script .py 

In [17]:
#!jupyter nbconvert --to script extraction_polyligne_TIM.ipynb

In [18]:
len(cache)

8190